In [1]:
import os
import sys
import uuid

pipeline_dir = os.path.join(os.getcwd(), '..')
if pipeline_dir not in sys.path:
    sys.path.append(pipeline_dir)

In [2]:
from pipeline import Pipeline, events, functions

# Functions
## Defining Functions
You can define a lambda function by declaring a method inside your `pipeline.Pipeline` object.  The method must be decorated with an event decorator.

In [3]:
class MyPipeline(Pipeline):
    
    def __init__(self):
        super().__init__(name="defining-functions")
    
    @events.invoke
    def lambda_func(self, event, context):
        print(event)

pipeline = MyPipeline()

You can use arguments to customize more complex events:

In [11]:
@events.http(method="get", path="todos", cors="true")
def lambda_func_http(self, event, context):
    print(event)

You can customize your function's runtime configuration with more decorators:

In [12]:
@events.invoke
@functions.timeout(30) # Specify function's timeout in seconds
@functions.memory(3008) # Specify function's memory limit in MB
def lambda_func(self, event, context):
    print(event)

## Writing Functions
#### Input Handling
The event decorator used by the pipeline method modifies the input event message.  For example, the `HTTP POST` decorator modifies the original message by calling `json.loads(event['body'])` and passing the resulting dictionary to your pipeline method.  This feature is intended for QOL, as input messaging parsing happens automatically and lambda functions.  You may use the `legacy` input parameter to disable this behavior.

In [13]:
@events.http(method="post", path="todos", cors="true")
def lambda_func(self, event, contet):
    assert type(event) == dict

@events.http(method="post", path="todos", cors="true", legacy=True)
def lambda_func(self, event, contet):
    assert type(event) == str

#### Output Handling
The output of your lambda function is determined by your pipeline method's return statement.  Please be careful to ensure that functions are returning the appropriate output for the event type.  For example, any HTTP response must be of the form `json.dumps({'statusCode': 200, 'body': 'return_message'})`

Not all functions require a return statement.  A lambda function which simply relays a message to a SNS Topic does not have a meaningful return.

#### Function Handling
When instantiated, the pipeline object stores all lambda functions in the `functions` attribute:

In [14]:
print(pipeline.functions.all)

{'lambda_func': <pipeline.functions.Function object at 0x7f9a8c238c50>}


You can access particular lambda functions using the method's name:

In [17]:
my_lambda = pipeline.functions['lambda_func']

Once your pipeline is deployed to AWS, you can invoke a particular lambda function using the function's `invoke` method to activate the lambda function's trigger.  The input to `invoke` changes based on the event type.

In [20]:
# This cell will fail (pipeline is not deployed to AWS)
try:
    my_lambda.invoke({'testing': '123'})
except:
    print("Invocation failed")
    pass

Invocation failed
